# First round queries

In [1]:
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
total_colums = [
    # Ejemplar
    'TITULO', 'AUTOR', 'EDITORIAL', 'SUCURSAL', 'DESC_SUCURSAL', 'COD_BARRAS_LIBRO', 'NUM_TITULO', 'SIG_TOPOGRAFICA',
    'DEWEY', 'II_SUMARIO', 'I_SUMARIO', 'EDICION', 'LOCALIZACION_EJEMPLAR', 'DESC_LOCALIZACION_EJEMPLAR', 'POLITICA',
    'DESC_POLITICA_EJEMPLAR', 'ISBN', 'FECHA_REGISTRO_EJEMPLAR', 'FECHA_ADICION_EJEMPLAR', 'FECHA_PUB',
    'ESTADO_EJEMPLAR', 'PAIS_EJEMPLAR',
    # Usuario (I)
    'OCUPACION', 'NIVEL_EDUCACION', 'AREA_ESTUDIO',
    # Prestamo (I)
    'SUCURSAL_PRESTAMO', 'SUCURSAL_DEVOLUCION',
    # Usuario (II)
    'SUCURSAL_DEL_LECTOR', 'COD_BARRAS_LECTOR', 'CATEGORÍA',
    # Prestamo (II)
    'FECHA_PRESTAMO', 'FECHA_REAL_DEVOLUCION',
]

In [3]:
used_colums = [
    # Ejemplar
    'TITULO', 'AUTOR', 'SUCURSAL', 'COD_BARRAS_LIBRO', 'DEWEY', 'DESC_LOCALIZACION_EJEMPLAR',
    'FECHA_PUB','PAIS_EJEMPLAR',
    # Usuario (I)
    'OCUPACION','NIVEL_EDUCACION','AREA_ESTUDIO',
    # Prestamo (I)
    'SUCURSAL_PRESTAMO','SUCURSAL_DEVOLUCION',
    # Usuario (II)
    'SUCURSAL_DEL_LECTOR', 'COD_BARRAS_LECTOR', 'CATEGORÍA',
    # Prestamo (II)
    'FECHA_PRESTAMO','FECHA_REAL_DEVOLUCION'
]

# Préstamos

In [4]:
%%time

dtypes = {
    'TITULO': str,
    'AUTOR': str,
    'SUCURSAL': str,
    'COD_BARRAS_LIBRO': str,
    'DEWEY': str,
    'DESC_LOCALIZACION_EJEMPLAR': str,
    'FECHA_PUB': str,
    'PAIS_EJEMPLAR': str,
    'OCUPACION': str,
    'NIVEL_EDUCACION': str,
    'AREA_ESTUDIO': str,
    'SUCURSAL_PRESTAMO': str,
    'SUCURSAL_DEVOLUCION': str,
    'SUCURSAL_DEL_LECTOR': str,
    'COD_BARRAS_LECTOR': str,
    'CATEGORÍA': str,
    'FECHA_PRESTAMO': str,
    'FECHA_REAL_DEVOLUCION': str,
    'DATE_PRESTAMO': pd.tslib.Timestamp,
    'DATE_DEVOLUCION': pd.tslib.Timestamp,
}

prestamos_df = pd.read_csv('prestamos_totales.csv', header=0, dtype=dtypes, parse_dates=['DATE_PRESTAMO', 'DATE_DEVOLUCION'])
prestamos_df = prestamos_df.fillna('')

CPU times: user 18.8 s, sys: 1.11 s, total: 19.9 s
Wall time: 19.9 s


In [5]:
prestamos_df.shape

(3054360, 20)

In [6]:
prestamos_df.head()

,TITULO,AUTOR,SUCURSAL,COD_BARRAS_LIBRO,DEWEY,DESC_LOCALIZACION_EJEMPLAR,FECHA_PUB,PAIS_EJEMPLAR,OCUPACION,NIVEL_EDUCACION,AREA_ESTUDIO,SUCURSAL_PRESTAMO,SUCURSAL_DEVOLUCION,SUCURSAL_DEL_LECTOR,COD_BARRAS_LECTOR,CATEGORÍA,FECHA_PRESTAMO,FECHA_REAL_DEVOLUCION,DATE_PRESTAMO,DATE_DEVOLUCION
0,Un bicho extraño,"Daporta, Mon.",ARM,2298061,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88429527,CTA,22/01/2015,04/02/2015,2015-01-22,2015-02-04
1,Un bicho extraño,"Daporta, Mon.",ARM,2298061,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88416381,CTA,31/03/2015,07/04/2015,2015-03-31,2015-04-07
2,Quisiera tener ...,"Zoboli, Giovanna.",ARM,2284458,853,Infantil,2010,mx,,,,ARM,ARM,ARM,88381353,CTA,07/11/2015,17/11/2015,2015-11-07,2015-11-17
3,Johannes Gutenberg,"Borràs Perelló, Lluís.",ARM,2288922,925,Infantil,2010,sp,Empleado,Técnico,Adm. de empresas,ARM,ARM,ARM,88007424,CEM,16/01/2015,16/01/2015,2015-01-16,2015-01-16
4,La fantástica leyenda de: la princesa y el dragón,"Alins, Sonia.",ARM,2288896,863,Infantil,2010,sp,,,,ARM,ARM,ARM,88429529,CTA,21/01/2015,28/01/2015,2015-01-21,2015-01-28


# Sucursales

In [7]:
sucursales = {
    'ARM': {'name': 'Armenia', 'latitude': 4.53, 'longitude': -75.68},
    'BAR': {'name': 'Barranquilla', 'latitude': 10.963889, 'longitude': -74.796389},
    'BLAA': {'name': 'Luis Angel Arango', 'latitude': 4.59677, 'longitude': -74.072848},
    'BUC': {'name': 'Bucaramanga', 'latitude': 7.133333, 'longitude': -73},
    'BUE': {'name': 'Buenaventura', 'latitude': 3.877222, 'longitude': -77.026667},
    'CAL': {'name': 'Cali', 'latitude': 3.420556, 'longitude': -76.522222},
    'CAR': {'name': 'Cartagena', 'latitude': 10.4, 'longitude': -75.5},
    'CASAGO': {'name': 'Casa Gómez Campuzano', 'latitude': 4.6628468, 'longitude': -74.0512116},
    'CATMAN': {'name': 'Catalogación y Mantenimiento de Colecciones', 'latitude': 4.5769749, 'longitude': -74.0880581},
    'CUC': {'name': 'Cúcuta', 'latitude': 7.894167, 'longitude': -72.503889},
    'ESPE': {'name': 'Especial material audiovisual', 'latitude': 4.59677, 'longitude': -74.072848},
    'FLO': {'name': 'Florencia', 'latitude': 1.614167, 'longitude': -75.611667},
    'GIR': {'name': 'Girardot', 'latitude': 4.305, 'longitude': -74.801667},
    'HON': {'name': 'Honda', 'latitude': 5.204167, 'longitude': -74.741667},
    'IBA': {'name': 'Ibagué', 'latitude': 4.433333, 'longitude': -75.233333},
    'IEC': {'name': 'Investigaciones Económicas', 'latitude': 4.5769749, 'longitude': -74.0880581},
    'IPI': {'name': 'Ipiales', 'latitude': 0.830278, 'longitude': -77.644444},
    'LET': {'name': 'Leticia', 'latitude': -4.205278, 'longitude': -69.932778},
    'MAN': {'name': 'Manizales', 'latitude': 5.1, 'longitude': -75.55},
    'MED': {'name': 'Medellín', 'latitude': 6.230833, 'longitude': -75.590556},
    'MON': {'name': 'Montería', 'latitude': 8.75, 'longitude': -75.883333},
    'NEI': {'name': 'Neiva', 'latitude': 2.998611, 'longitude': -75.304444},
    'PAS': {'name': 'Pasto', 'latitude': 1.207778, 'longitude': -77.277222},
    'PER': {'name': 'Pereira', 'latitude': 4.814278, 'longitude': -75.694558},
    'POP': {'name': 'Popayán', 'latitude': 2.454167, 'longitude': -76.609167},
    'QUI': {'name': 'Quibdó', 'latitude': 5.692222, 'longitude': -76.658056},
    'RIO': {'name': 'Riohacha', 'latitude': 11.544167, 'longitude': -72.906944},
    'SAN': {'name': 'San Andrés', 'latitude': 12.583333, 'longitude': -81.7},
    'SIN': {'name': 'Sincelejo', 'latitude': 9.295, 'longitude': -75.396111},
    'STM': {'name': 'Santa Marta', 'latitude': 11.241944, 'longitude': -74.205278},
    'TUN': {'name': 'Tunja', 'latitude': 5.533333, 'longitude': -73.366667},
    'VAL': {'name': 'Valledupar', 'latitude': 10.483333, 'longitude': -73.25},
    'VIL': {'name': 'Villavicencio', 'latitude': 4.15, 'longitude': -73.633333},
}

# Country codes

## LOC/MARC

http://www.loc.gov/marc/countries/countries_code.html

In [8]:
loc_marc_df = pd.read_csv('codes-loc-marc-countries.csv', sep='\t', names=['code','country'])
loc_marc_sr = pd.Series(data=list(loc_marc_df['country']), index=list(loc_marc_df['code']))

In [9]:
loc_marc_sr.head()

aa                          Albania
abc                         Alberta
-ac     Ashmore and Cartier Islands
aca    Australian Capital Territory
ae                          Algeria
dtype: object

## ISO

http://www.nationsonline.org/oneworld/country_code_list.htm

In [10]:
iso_df = pd.read_csv('codes-iso-countries.csv', sep='\t', names=['country', 'iso-2', 'iso-3', 'code'])
iso2_sr = pd.Series(data=list(iso_df['country']), index=map(lambda x: str(x).lower(), iso_df['iso-2']))
iso3_sr = pd.Series(data=list(iso_df['country']), index=map(lambda x: str(x).lower(), iso_df['iso-3']))

In [11]:
iso2_sr.head()

af       Afghanistan
ax     Aland Islands
al           Albania
dz           Algeria
as    American Samoa
dtype: object

In [12]:
iso3_sr.head()

afg       Afghanistan
ala     Aland Islands
alb           Albania
dza           Algeria
asm    American Samoa
dtype: object

## Mine

In [13]:
missing_codes_dict = {
    'uk': 'United Kingdom',
    'en': 'England',
    'spa': 'Spain',
    'CK': 'Colombia',
    'eng': 'England',
    'ger': 'Germany',
    'uy|': 'Uruguay',
    'SP': 'Spain',
    'AG': 'Argentina',
    'por': 'Portugal',
    'jap': 'Japan',
    'ur': 'Uruguay',
    'fl': 'Florida',
    'flo': 'Florida',
    'fre': 'French Guiana',
    'he': 'Heard Island and Mcdonald Islands',
    'ka': 'Kansas',
    'lou': 'Louisiana',
    'pu': 'Puerto Rico',
    'wa': 'Washington (State)',
}

# Autores

In [14]:
autores_df = pd.DataFrame()

## Título

In [15]:
autores_df['TITULO'] = prestamos_df['TITULO']

## Año publicación

In [16]:
autores_df['FECHA_PUB'] = prestamos_df['FECHA_PUB']

## País ejemplar

In [17]:
prestamos_df['PAIS_EJEMPLAR'].value_counts()

sp     1194374
ck      968489
mx      366393
ag      180189
???      72351
us       46255
ve       28224
enk      19247
cl       17621
nyu      15367
fr       12416
gx       10072
cau       9985
pe        9473
ec        9341
bl        8413
en        7996
cu        7249
it        6253
gw        5767
uk        5555
xxu       3088
cc        2974
mdu       2566
uy        2397
cr        2313
spa       2235
          1850
ur        1794
mau       1623
        ...   
hn           1
quc          1
bcc          1
ae           1
tr           1
unk          1
as           1
fn           1
mtu          1
f            1
us-          1
spy          1
ev           1
ka           1
hi           1
utu          1
wa           1
v            1
lou          1
efr          1
sus          1
ssp          1
fs           1
cii          1
aut          1
nt           1
bh           1
ce           1
zz           1
sbe          1
Name: PAIS_EJEMPLAR, dtype: int64

In [18]:
total_countries = set(list(loc_marc_sr.values) + list(iso2_sr.values) + list(iso3_sr.values))

In [19]:
%%time

def process_pais(x):
    a = x['PAIS_EJEMPLAR'].strip()
    if a in loc_marc_sr:
        r = loc_marc_sr[a]
    elif '-' + a in loc_marc_sr:
        r = loc_marc_sr['-' + a]
    elif a in iso3_sr:
        r = iso3_sr[a]
    elif a in iso2_sr:
        r = iso2_sr[a]
    elif a in missing_codes_dict:
        r = missing_codes_dict[a]
    else:
        r = 'DESCONOCIDO'
    return r

autores_df['PAIS_EJEMPLAR'] = prestamos_df.apply(process_pais, axis=1)

CPU times: user 7min 59s, sys: 1.04 s, total: 8min
Wall time: 7min 59s


In [20]:
autores_df['PAIS_EJEMPLAR'].value_counts()

Spain                                          1196618
Colombia                                        968719
Mexico                                          366627
Argentina                                       180773
DESCONOCIDO                                      86413
United States                                    49631
Venezuela                                        28224
England                                          27375
Chile                                            17621
New York (State)                                 15367
France                                           12416
California                                        9985
Peru                                              9473
Ecuador                                           9359
Brazil                                            8413
Cuba                                              7249
United Kingdom                                    6916
Italy                                             6338
Germany   

## Nombre

In [21]:
prestamos_df['AUTOR'].nunique()

148860

In [22]:
prestamos_df['AUTOR'].value_counts()

                                                                409373
García Márquez, Gabriel, 1927-2014                               13207
Colombia.                                                        11526
Hergé, 1907-1983.                                                11510
Browne, Anthony, 1946-.                                           8653
Foucault, Michel, 1926-1984.                                      6849
Cortázar, Julio, 1914-1984.                                       6478
Nietzsche, Friedrich Wilhelm, 1844-1900.                          5876
Da Coll, Ivar, 1962-                                              5350
Kasza, Keiko.                                                     5336
Shakespeare, William, 1564-1616.                                  5012
Verne, Jules, 1828-1905.                                          4394
Instituto Colombiano de Normas Técnicas y Certificación.          4318
Dostoievski, Fedor Mijailovich, 1821-1881.                        4296
Rowlin

In [23]:
%%time

def process_autor(x):
    a = x['AUTOR'].strip(' ' + punctuation)
    a = a.replace(', ', ',')
    if a:
        b = re.findall('(.+),\d+', a)
        if b:
            a = b[0]
        c = a.split(',')
        if len(c) > 1:
            r = c[1] + ' ' + c[0]
        else:
            r = c[0]
    else:
        r = 'DESCONOCIDO'
    return r

autores_df['AUTOR'] = prestamos_df.apply(process_autor, axis=1)

CPU times: user 7min 37s, sys: 1.16 s, total: 7min 38s
Wall time: 7min 37s


In [24]:
autores_df['AUTOR'].nunique()

145132

In [25]:
autores_df['AUTOR'].value_counts()

DESCONOCIDO                                                     409373
Gabriel García Márquez                                           14397
Colombia                                                         11526
Hergé                                                            11510
Anthony Browne                                                    9653
Michel Foucault                                                   6849
Julio Cortázar                                                    6478
Friedrich Wilhelm Nietzsche                                       5876
Ivar Da Coll                                                      5350
Keiko Kasza                                                       5336
William Shakespeare                                               5012
José Saramago                                                     4818
Jules Verne                                                       4394
Instituto Colombiano de Normas Técnicas y Certificación           4333
Fedor 

## País autor

In [26]:
def get_pais_autor_data_wkd(name, lang):
    template = """
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT DISTINCT ?authorLabel ?countryLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author wdt:P31 wd:Q5 .
            {{
                {{
                    ?author wdt:P106 wd:Q36180 .
                }}
                UNION
                {{ 
                    ?author wdt:P106 ?prof .
                    ?prof wdt:P279 wd:Q36180 .
                }}
            }}
            ?author wdt:P19 ?pob .
            ?pob wdt:P17 ?country .
            ?country rdfs:label ?countryLabel .
            FILTER(lang(?countryLabel)="{1}")
        }}
    """
    query = template.format(name, lang)
    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    data = requests.get(url, params={'query': query, 'format': 'json'}).json()
    return data

In [27]:
sparql_dbp = SPARQLWrapper("http://dbpedia.org/sparql")

def get_pais_autor_data_dbp(name, lang):
    template = """
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX dbp: <http://dbpedia.org/property/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT DISTINCT ?authorLabel ?countryLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author rdf:type dbo:Person .
            {{
                {{
                    ?author rdf:type dbo:Writer .
                }}
                UNION
                {{
                    ?author rdf:type ?prof .
                    ?prof rdfs:subClassOf dbo:Writer .
                }}
            }}
            ?author dbp:birthPlace ?pob .
            ?pob rdf:type dbo:Country .
            ?pob rdfs:label ?countryLabel .
            FILTER(lang(?countryLabel)="{1}")
        }}
    """
    query = template.format(name, lang)
    sparql_dbp.setQuery(query)
    sparql_dbp.setReturnFormat(JSON)
    data = sparql_dbp.query().convert()
    return data

In [28]:
def get_pais_autor_data(db, name, lang):
    try:
        if db == 'wdt':
            data = get_pais_autor_data_wkd(name, lang)
        elif db == 'dbp':
            data = get_pais_autor_data_dbp(name, lang)

        results = set(d['countryLabel']['value'] for d in data['results']['bindings'])
        if len(results) == 0:
            result = 'UNKOWN'
        elif len(results) == 1:
            result = results.pop()
        else:
            print('MáS DE UN PAíS PARA', name, ':', results)
            result = results.pop()
    except:
        print('ERROR EN SPARQL PARA', name)
        result = 'UNKOWN'
    return result

In [29]:
def get_pais_autor(name):
    for lang in ['en', 'es']:
        for db in ['wdt', 'dbp']:
            r = get_pais_autor_data(db, name, lang)
            if r != 'UNKOWN':
                return r
    return 'DESCONOCIDO'

In [30]:
%%time

autores_paises_dict = {}

for x in autores_df['AUTOR'].unique():
    autores_paises_dict[x] = get_pais_autor(x)

MáS DE UN PAíS PARA Ludmila Zeman : {'Czech Republic', 'Czechoslovakia'}
MáS DE UN PAíS PARA David Shannon : {'United States of America', 'United Kingdom'}
MáS DE UN PAíS PARA Mario Ramos : {'Spain', 'Belgium'}
MáS DE UN PAíS PARA Mario Benedetti : {'Uruguay', 'Italy'}
ERROR EN SPARQL PARA Instituto Geográfico "Agustín Codazzi". Subdirección de Inve
ERROR EN SPARQL PARA Instituto Geográfico "Agustín Codazzi". Subdirección de Inve
ERROR EN SPARQL PARA Instituto Geográfico "Agustín Codazzi". Subdirección de Inve
ERROR EN SPARQL PARA Instituto Geográfico "Agustín Codazzi". Subdirección de Inve
MáS DE UN PAíS PARA Yotam Ottolenghi : {'Palestine', 'Israel', 'Ottoman Empire', 'Jordan', 'Mandatory Palestine'}
ERROR EN SPARQL PARA Encuentro Internacional "La Economía de los Trabajadores" (2
ERROR EN SPARQL PARA Encuentro Internacional "La Economía de los Trabajadores" (2
ERROR EN SPARQL PARA Encuentro Internacional "La Economía de los Trabajadores" (2
ERROR EN SPARQL PARA Encuentro Internacion

In [31]:
%%time

def process_pais(x):
    return autores_paises_dict[x['AUTOR']]

autores_df['PAIS_AUTOR'] = autores_df.apply(process_pais, axis=1)

CPU times: user 1min 25s, sys: 0 ns, total: 1min 25s
Wall time: 1min 25s


# Saving results

In [32]:
autores_df.to_csv('autores1.csv', index=False)